# RDD

En visión general, un RDD puede ser visto como un set de datos los cuales soportan solo dos tipos de operaciones: transformaciones y acciones.

Las transformaciones permiten crear un nuevo RDD a partir de uno previamente existente, mientras que las acciones retornan un valor al driver de la aplicación. El núcleo de operación del paradigma de Spark es la ejecución perezosa (Lazy), es decir que las transformaciones solo serán calculadas posterior a una llamada de acción.

Además, los RDD poseen una familiaridad con el paradigma orientado a objetos, lo cual permite que podamos realizar operaciones de bajo nivel a modo. Map, filter y reduce son tres de las operaciones más comunes.

Una de las grandes ventajas que ofrecen los RDD es la compilación segura; por su particularidad de ejecución perezosa, se calcula si se generará un error o no antes de ejecutarse, lo cual permite identificar problemas antes de lanzar la aplicación. El pero que podemos encontrar con los RDD es que no son correctamente tratados por el Garbage collector y cuando las lógicas de operación se hacen complejas, su uso puede resultar poco práctico, aquí entran los DataFrames.

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local", appName = "Transformaciones y Acciones")

In [3]:
rdd1 = sc.parallelize([1,2,3])

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=Transformaciones y Acciones>

In [6]:
!ls ./data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [8]:
#Creamos path para facilitar uso de archivos
path = "./data/"

In [9]:
equiposOlimpicosRDD = sc.textFile(path+"paises.csv") \
    .map(lambda line : line.split(","))

### Take:
- Me muestra la cantidad de lineas que le pasemos
* No podemos ver acciones o si hay error hasta usar una ACCION

In [10]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [11]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

# Los RDD son inmutables, realizar estas acciones no genera cambios en 
# RDD , pero puedo re asignar el resultado a una variable nueva

231

Nos encontramos con 231 paises participando en los JJOO, incluyendo paises extintos como la Union Sovietica

In [12]:
#Funcionan de la forma llave, valor , motivo por esto invertimos orden
equiposOlimpicosRDD.map(lambda x :  (x[2], x[1] )).groupByKey() \
    .mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
equiposOlimpicosRDD.map(lambda x :  (x[2], x[1] )).groupByKey() \
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [14]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()
#Collect manda todos, por eso no es recomendado usarlo.

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [15]:
equiposColombianos = equiposOlimpicosRDD.filter(lambda l : "COL" in l)
equiposColombianos.collect()

[['225', 'Colombia', 'COL']]

In [16]:
equiposOlimpicosRDD.count() #Cuantos valores tiene este RDD? Easy!

1185

In [17]:
equiposOlimpicosRDD.countApprox(20) #Cuenta cuantos valores consigue leer en 
# la cantidad de tiempo indicada

1185

In [18]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv") \
    .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path+"deportista2.csv") \
    .map(lambda l: l.split(","))

# Joining and Unions

Unimos los dos RDD para no tener más duplicados:

In [19]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

In [20]:
deportistaOlimpicoRDD.count()

135572

In [21]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [22]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

Observemos como id y equipo id son el valor que nos facilitan el JOIN

In [23]:
deportistaOlimpicoRDD.map(lambda l : [l[-1],l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])) \
    .take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [24]:
deportistaOlimpicoRDD.map(lambda l : [l[-1],l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]])) \
    .takeSample(False, 6, 25)
#Take sample: muestras aleatorias (repetidos, cantidad, semilla)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [25]:
resultado = sc.textFile(path+"resultados.csv") \
    .map(lambda l : l.split(","))

In [26]:
resultado.take(1)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id']]

In [27]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1]) #Quitamos los que no ganaron

In [28]:
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [29]:
# Reto. Hacer Join con el RDD anterior
deportistaOlimpicoRDD.take(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199']]

In [30]:
equiposOlimpicosRDD.take(2)

[['id', 'equipo', 'sigla'], ['1', '30. Februar', 'AUT']]

In [31]:
deportista_paises = deportistaOlimpicoRDD.map(lambda l : [l[-1], l[:-1]]) \
        .join(equiposOlimpicosRDD.map(lambda x : [x[0],x[2]]))

In [32]:
deportista_paises.take(2)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [33]:
#Primer map: Extraer jugador id y el resto de valores(key,values)
#Segundo map: Extraer jugador id y valor de medalla (Adicional innecesario)
paises_medallas = deportista_paises.map(lambda l : (l[1][0][0] , (l[0] , l[1][0][1:],l[1][1]),l[1][1] )) \
        .join(resultadoGanador.map(lambda l : (l[2],l[1])))

In [34]:
paises_medallas.takeSample(False, 5, 11)

[('12941',
  (('1096', ['Jeanette Bolden', '2', '24', '174', '65'], 'USA'), 'Gold')),
 ('9803',
  (('249', ['Daima Mayelis Beltrn Guisado', '2', '28', '182', '104'], 'CUB'),
   'Silver')),
 ('77779',
  (('1096', ['Jeffrey Dean Jeff McLaughlin', '1', '22', '198', '91'], 'USA'),
   'Bronze')),
 ('89894',
  (('1084',
    ['Inna Volodymyrivna OsypenkoRadomska', '2', '17', '166', '63'],
    'UKR'),
   'Gold')),
 ('74240',
  (('535', ['Aset Sekenovich Mambetov', '1', '22', '186', '96'], 'KAZ'),
   'Bronze'))]

In [35]:
valores_medallas = {
    'Gold' : 7,
    'Silver' : 5,
    'Bronze' : 4
}

In [36]:
paises_medallas.map(lambda x : (x[1][0][2], valores_medallas[x[1][1]])).take(5)

[('CHN', 5), ('CHN', 4), ('CHN', 5), ('CHN', 5), ('CHN', 5)]

In [37]:
paises_medallas = paises_medallas.map(lambda x : (x[1][0][2], valores_medallas[x[1][1]]))

# Agrupar en un RDD
Debido a lo primitivo que son los RDD ocupados realizar operaciones de bajo nivel

In [38]:
from operator import add

In [39]:
conclusion = paises_medallas \
    .reduceByKey((add)) \
    .sortBy(lambda x : x[1], ascending=False)

In [40]:
conclusion.take(10)
# (pais, puntaje)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [41]:
#sc.stop()